First, let's import all the necessary dependencies.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

Let's define some configuration variables.

In [ ]:
DATASET = "s3://hedp/ecmwf-forecasts/opendata-oper-20231016T00.zarr"
LOCATION = {"latitude": 41.9, "longitude": 12.5}

In the following lines we:
- establish a connection with the remote dataset
- show its metadata

It is worth noticing that, in the first operation, only the information necessary to gather the dataset's metadata (e.g. its sizes, dimensions, variables' type and attributes) is actually downloaded from its remote location to the local machine.

In [ ]:
ds = xr.open_dataset(DATASET, chunks={}, engine="zarr").astype("float32")
ds

Here we want to extract the 2-metre Temperature forecast, in Celsius, for a specific location.

Note that the operations prescribed in the following lines are actually performed, and the resulting data downloaded, only when the `.compute()` method is met. For this reason, as a general rule to maximize efficiency, the `compute()` method should be placed after an operation which significantly reduce the amount of data, such as after selection and reduction operations.

In [ ]:
%%time

t2m = ds["2t"] - 273.15
t2m["step"] = t2m.step.astype('datetime64[ns]').astype('float64') / 1e9 / 3600
t2m.attrs["units"] = "C"
t2m_location = t2m.sel(**LOCATION, method="nearest")
t2m_location

In [ ]:
%%time
%%capture

t2m_location = t2m_location.compute()

Finally, let's plot the results on a time series.

In [ ]:
t2m_location.plot()
plt.title("Surface Temperature forecast in Rome (IT)")
plt.ylabel("Surface Temperature [C]")
reference_time = np.datetime_as_string(t2m_location.time.values)
plt.xlabel(f"time since {reference_time[:-10]} [hr]")
plt.show()